In [1]:
import random

import weaviate
import weaviate.classes as wvc
from weaviate.classes.query import HybridFusion
import os
import requests
import json
import cohere
import csv
import anthropic
import time
import pickle

from dotenv import load_dotenv

load_dotenv()

cohere_api_key = os.getenv('COHERE_PROD')
co = cohere.Client(api_key=cohere_api_key, log_warning_experimental_features=False)


wcd_url = os.getenv('WEAVIATE_URL')
wcd_api_key = os.getenv('WEAVIATE_API')



/home/amin/.pyenv/versions/finalproject/lib/python3.12/site-packages/google/protobuf/runtime_version.py:112: UserWarning: Protobuf gencode version 5.27.2 is older than the runtime version 5.28.0 at grpc_health/v1/health.proto. Please avoid checked-in Protobuf gencode that can be obsolete.
  warnings.warn(
/home/amin/.pyenv/versions/finalproject/lib/python3.12/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/home/amin/.pyenv/versions/finalproject/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)
/hom

In [2]:
with weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,
    auth_credentials=wvc.init.Auth.api_key(wcd_api_key),
    headers={"X-Cohere-Api-Key": cohere_api_key},
    skip_init_checks=True
) as client:

    book_collection = client.collections.get('OpenITI')
    response = book_collection.query.hybrid(
    query="حديث",
    alpha=0.1,
    limit=3,
)

for o in response.objects:
    print(o.properties)


{'volume': 'V03', 'text': 'حظه من خير الدنيا والآخرة حديث يا عائشة إنه من أعطى حظه من الرفق فقد أعطى حظه من خير الدنيا والآخرة الحديث رواه أحمد والعقيلي في الضعفاء في ترجمة عبد الرحمن بن أبي بكر المليكي وضعفه عن القاسم عن عائشة وفي الصحيحين من حديثها يا عائشة إن الله يحب الرفق في الأمر كله وقال صلى الله عليه وسلم إذا أحب الله أهل بيت أدخل عليهم الرفق حديث إذا أحب الله أهل بيت أدخل عليهم الرفق أخرجه أحمد بسند جيد والبيهقي في الشعب بسند ضعيف من حديث عائشة وقال صلى الله عليه وسلم إن الله ليعطي على الرفق ما لا يعطي على الخرق وإذا أحب الله عبدا أعطاه الرفق وما من أهل بيت يحرمون الرفق إلا حرموا محبة الله تعالى حديث إن الله ليعطي على الرفق ما لا يعطي على الخرق الحديث أخرجه الطبراني في الكبير من حديث جرير بإسناد ضعيف وقالت عائشة رضي الله عنها قال النبي صلى الله عليه وسلم إن الله رفيق يحب الرفق ويعطي عليه ما لا يعطي على العنف حديث إن الله رفيق يحب الرفق الحديث أخرجه مسلم من حديث عائشة وقال صلى الله عليه وسلم يا عائشة ارفقي فإن الله إذا أراد بأهل بيت كرامة دلهم على باب الرفق حديث يا عائشة ارفقي 

In [4]:
with weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,
    auth_credentials=wvc.init.Auth.api_key(wcd_api_key),
    headers={"X-Cohere-Api-Key": cohere_api_key},
    skip_init_checks=True
) as client:

    book_collection = client.collections.get('OpenITI')
    response = book_collection.query.near_text(
    query="What is the importance of prayer in islam?",
    limit=2,
    return_metadata=wvc.query.MetadataQuery(distance=True)
)

# Extract the results for reranking
results = []
for o in response.objects:
    results.append({
        "text": o.properties['text'],  # Assuming 'text' is the field in your collection
        "uuid": o.uuid
    })

print("Initial Weaviate Results:")
for result in results:
    print(result['text'])

Initial Weaviate Results:
بالدعاء المأثور مع التواضع والخشوع والضراعة والابتهال وصدق الرجاء بالإجابة وأشرك في دعائك أبويك وسائر المؤمنين واقصد عند التسليم السلام على الملائكة والحاضرين وانو ختم الصلاة به واستشعر شكر الله سبحانه على توفيقه لإتمام هذه الطاعة وتوهم أنك مودع لصلاتك هذه وأنك ربما لا تعيش لمثلها وقال صلى الله عليه وسلم للذي أوصاه صل صلاة مودع ثم أشعر قلبك الوجل والحياء من التقصير في ا الصلاة وخف أن لا تقبل صلاتك وأن تكون ممقوتا بذنب ظاهر أو باطن فترد صلاتك في وجهك وترجو مع ذلك أن يقلبها بكرمه وفضله كان يحيى بن وثاب إذا صلى مكث ما شاء الله تعرف عليه كآبة الصلاة وكان إبراهيم يمكث بعد الصلاة ساعة كأنه مريض فهذا تفصيل صلاة الخاشعين الذين هم في صلاتهم خاشعون والذين هم على صلواتهم يحافظون والذين هم على صلاتهم دائمون والذين هم يناجون الله على قدر استطاعتهم في العبودية فليعرض الإنسان نفسه على هذه الصلاة فبالقدر الذي يسر له منه ينبغي أن يفرح وعلى ما يفوته ينبغي أن يتحسر وفي مداراة ذلك ينبغي أن يجتهد وأما صلاة الغافلين فهي مخطرة إلا أن يتغمده الله برحمته والرحمة واسعة والكرم فائض فنسأ

In [5]:
response.objects

[Object(uuid=_WeaviateUUIDInt('0b46c3b1-807e-4c7a-9edd-6d1278246f23'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=0.4223760962486267, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'volume': 'V01', 'text': 'بالدعاء المأثور مع التواضع والخشوع والضراعة والابتهال وصدق الرجاء بالإجابة وأشرك في دعائك أبويك وسائر المؤمنين واقصد عند التسليم السلام على الملائكة والحاضرين وانو ختم الصلاة به واستشعر شكر الله سبحانه على توفيقه لإتمام هذه الطاعة وتوهم أنك مودع لصلاتك هذه وأنك ربما لا تعيش لمثلها وقال صلى الله عليه وسلم للذي أوصاه صل صلاة مودع ثم أشعر قلبك الوجل والحياء من التقصير في ا الصلاة وخف أن لا تقبل صلاتك وأن تكون ممقوتا بذنب ظاهر أو باطن فترد صلاتك في وجهك وترجو مع ذلك أن يقلبها بكرمه وفضله كان يحيى بن وثاب إذا صلى مكث ما شاء الله تعرف عليه كآبة الصلاة وكان إبراهيم يمكث بعد الصلاة ساعة كأنه مريض فهذا تفصيل صلاة الخاشعين الذين هم في صلاتهم خاشعون والذين هم على صلواتهم يحافظون والذين هم على صلاتهم دائمون وا

In [27]:
from weaviate.classes.query import Rerank, MetadataQuery, Filter

with weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,
    auth_credentials=wvc.init.Auth.api_key(wcd_api_key),
    headers={"X-Cohere-Api-Key": cohere_api_key},
    skip_init_checks=True
) as client:

    book_collection = client.collections.get('OpenITI')
    response = book_collection.query.near_text(
    query="what is the importance of prayer in islam?",
    limit=10,
    rerank=Rerank(
        prop="text",
        query="according to the quran",
    ),
    return_metadata=MetadataQuery(score=True, distance=True),
    # filters=Filter.by_property("topic").equal("Gratitude"),
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.rerank_score)
    print(o.metadata.distance)


{'text': 'واقد رأيت ابن عمر يصلي مغلوبا عليه وحق له أن يحترق قلبه بوعد سيده ووعيده فإنه عبد مذنب ذليل بين يدي جبار قاهر وتكون هذه المعاني بحسب درجات الفهم ويكون الفهم بحسب وفور العلم وصفاء القلب ودرجات ذلك لا تنحصر والصلاة مفتاح القلوب فيها تنكشف أسرار الكلمات فهذا حق القراءة وهو حق الأذكار والتسبيحات أيضا ثم يراعى الهيبة في ا لقراءة فيرتل ولا يسرد فإن ذلك أيسر للتأمل ويفرق بين نغماته في آية الرحمة والعذاب والوعد والوعيد والتحميد والتعظيم والتمجيد كان النخعي إذا مر بمثل قوله عز وجل ما اتخذ الله من ولد وما كان معه من إله يخفض صوته كالمستحيي عن أن يذكره بكل شيء لا يليق به وروي أنه يقال لقارىء القرآن إقرأ وارق ورتل كما كنت ترتل في الدنيا حديث يقال لصاحب القرآن اقرأ وارق الحديث أخرجه أبو داود والترمذي والنسائي من حديث عبد الله بن عمر وقال الترمذي حسن صحيح وأما دوام القيام فإنه تنبيه على إقامة القلب مع الله عز وجل على نعت واحد من الحضور قال صلى الله عليه وسلم إن الله عز وجل مقبل على المصلي ما لم يلتفت إن الله يقبل على المصلي ما لم يلتفت أخرجه أبو داود والنسائي والحاكم وصحح إسناده أبي ذر وكم

In [9]:
response.objects

[Object(uuid=_WeaviateUUIDInt('87affeaa-6307-4717-93b7-3c2666d6521f'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=0.0, explain_score=None, is_consistent=None, rerank_score=0.09551807), properties={'text': 'واقد رأيت ابن عمر يصلي مغلوبا عليه وحق له أن يحترق قلبه بوعد سيده ووعيده فإنه عبد مذنب ذليل بين يدي جبار قاهر وتكون هذه المعاني بحسب درجات الفهم ويكون الفهم بحسب وفور العلم وصفاء القلب ودرجات ذلك لا تنحصر والصلاة مفتاح القلوب فيها تنكشف أسرار الكلمات فهذا حق القراءة وهو حق الأذكار والتسبيحات أيضا ثم يراعى الهيبة في ا لقراءة فيرتل ولا يسرد فإن ذلك أيسر للتأمل ويفرق بين نغماته في آية الرحمة والعذاب والوعد والوعيد والتحميد والتعظيم والتمجيد كان النخعي إذا مر بمثل قوله عز وجل ما اتخذ الله من ولد وما كان معه من إله يخفض صوته كالمستحيي عن أن يذكره بكل شيء لا يليق به وروي أنه يقال لقارىء القرآن إقرأ وارق ورتل كما كنت ترتل في الدنيا حديث يقال لصاحب القرآن اقرأ وارق الحديث أخرجه أبو داود والترمذي والنسائي من حديث عبد الله بن عمر وقا

In [29]:
from weaviate.classes.query import Rerank, MetadataQuery, Filter

with weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,
    auth_credentials=wvc.init.Auth.api_key(wcd_api_key),
    headers={"X-Cohere-Api-Key": cohere_api_key},
    skip_init_checks=True
) as client:

    book_collection = client.collections.get('OpenITI')
    config = book_collection.config.get()
    print(config.properties)


[_Property(name='text', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none'), _Property(name='page', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none'), _Property(name='volume', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none'), _Property(name='author', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vec

In [41]:
from weaviate.classes.query import Rerank, MetadataQuery, Filter
from weaviate.classes.aggregate import GroupByAggregate

with weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,
    auth_credentials=wvc.init.Auth.api_key(wcd_api_key),
    headers={"X-Cohere-Api-Key": cohere_api_key},
    skip_init_checks=True,
) as client:

    book_collection = client.collections.get("OpenITI")

    response = book_collection.aggregate.over_all(
        group_by=GroupByAggregate(
            prop="book",
        )
    )
    response2 = book_collection.aggregate.over_all(
        group_by=GroupByAggregate(
            prop="topic",
        )
    )

In [42]:
[group.grouped_by.value for group in response.groups]

['إحياء علوم الدين']

In [44]:
[group.grouped_by.value for group in response2.groups]

['Spirituality',
 'Virtue',
 'Faith',
 'Theology',
 'Afterlife',
 'Gratitude',
 'Rituals',
 'Wisdom',
 'Ethics',
 'Discipline']

In [67]:
from weaviate.classes.query import Rerank, MetadataQuery, Filter

with weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,
    auth_credentials=wvc.init.Auth.api_key(wcd_api_key),
    headers={"X-Cohere-Api-Key": cohere_api_key},
    skip_init_checks=True
) as client:

    book_collection = client.collections.get('OpenITI')
    response = book_collection.query.near_text(
    query="what is the importance of prayer in islam?",
    limit=5,
    return_metadata=MetadataQuery(score=True, distance=True),
    filters=
        Filter.all_of(
            [
                Filter.by_property("topic").contains_any(["Rituals"]),
                Filter.by_property("book").contains_any(["إحياء علوم الدين"]),
            ]
        )

)

for o in response.objects:
    print(o.properties)
    print(o.metadata.rerank_score)
    print(o.metadata.distance)


{'text': 'بالدعاء المأثور مع التواضع والخشوع والضراعة والابتهال وصدق الرجاء بالإجابة وأشرك في دعائك أبويك وسائر المؤمنين واقصد عند التسليم السلام على الملائكة والحاضرين وانو ختم الصلاة به واستشعر شكر الله سبحانه على توفيقه لإتمام هذه الطاعة وتوهم أنك مودع لصلاتك هذه وأنك ربما لا تعيش لمثلها وقال صلى الله عليه وسلم للذي أوصاه صل صلاة مودع ثم أشعر قلبك الوجل والحياء من التقصير في ا الصلاة وخف أن لا تقبل صلاتك وأن تكون ممقوتا بذنب ظاهر أو باطن فترد صلاتك في وجهك وترجو مع ذلك أن يقلبها بكرمه وفضله كان يحيى بن وثاب إذا صلى مكث ما شاء الله تعرف عليه كآبة الصلاة وكان إبراهيم يمكث بعد الصلاة ساعة كأنه مريض فهذا تفصيل صلاة الخاشعين الذين هم في صلاتهم خاشعون والذين هم على صلواتهم يحافظون والذين هم على صلاتهم دائمون والذين هم يناجون الله على قدر استطاعتهم في العبودية فليعرض الإنسان نفسه على هذه الصلاة فبالقدر الذي يسر له منه ينبغي أن يفرح وعلى ما يفوته ينبغي أن يتحسر وفي مداراة ذلك ينبغي أن يجتهد وأما صلاة الغافلين فهي مخطرة إلا أن يتغمده الله برحمته والرحمة واسعة والكرم فائض فنسأل الله أن يتغمدن

In [71]:
from weaviate.classes.query import Rerank, MetadataQuery, Filter

with weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,
    auth_credentials=wvc.init.Auth.api_key(wcd_api_key),
    headers={"X-Cohere-Api-Key": cohere_api_key},
    skip_init_checks=True
) as client:

    book_collection = client.collections.get('OpenITI')
    response = book_collection.generate.near_text(
    query="test",
    limit=2,
    grouped_task='provide a summary of the results in english',
    grouped_properties=['text'],
    rerank=Rerank(
        prop="text",
        query="test",
    ),
    return_metadata=MetadataQuery(score=True, distance=True),
    # filters=Filter.by_property("topic").equal("Gratitude"),
)

if response.generated:
    print(response.generated)
for o in response.objects:
    print(o.properties)
    print(o.metadata.rerank_score)
    print(o.metadata.distance)


The text provides a set of guidelines for self-examination and the identification of hidden character flaws. The first test mentioned is to engage in a discussion with a peer, and observe one's reaction when truth is spoken by the other person. If one feels reluctant to accept, acknowledge, and express gratitude for the other's insight, it indicates the presence of hidden arrogance, which should be addressed through both intellectual understanding and practical actions. The second test involves mingling with peers in gatherings, intentionally taking a lower position, and observing one's comfort level. If one feels uncomfortable, it suggests the presence of arrogance, which can be overcome by consistently practicing humility until it feels natural. The passage also warns against the deception of the ego, where one might consider sitting among shoes or distancing oneself from peers as humility, while it is, in fact, a manifestation of arrogance. 

The text further emphasizes the importan